In [31]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import colors, gridspec
import seaborn as sns
import visualize as vis
import preprocessing as pp
import calculate as cc
from read_WDF_class import WDF


folder = "/data/2021-10-03-EcoleVerreEtDiffusion/"
filename = "exampleM2.wdf"
filepath = os.path.join(folder, filename)

In [32]:
fff = WDF(filepath, verbose=True)

Reading the file: "exampleM2.wdf"


================================= Block : WDF1 =================================
size: 512, offset: 0
WdfFlag--------------------------------- : 	WdfXYXY
PointsPerSpectrum----------------------- : 	1015
Capacity-------------------------------- : 	68134
Count----------------------------------- : 	68134
AccumulationCount----------------------- : 	1
YlistLength----------------------------- : 	1
XlistLength----------------------------- : 	1015
DataOriginCount------------------------- : 	5
ApplicationName------------------------- : 	WiRE
ApplicationVersion---------------------- : 	5.1.0 build 8867
ScanType-------------------------------- : 	StreamLine
MeasurementType------------------------- : 	Map
StartTime------------------------------- : 	Thu Oct 17 15:08:26 2019
EndTime--------------------------------- : 	Fri Oct 18 05:35:34 2019
SpectralUnits--------------------------- : 	Counts
LaserWaveLength------------------------- : 	514.38
Title----------------

### Here are some of the treatments you might apply to your input spectra:
Do this first:
```python
fff = pp.order(fff)
```
Normalizing:
```python
fff = pp.normalize(fff)
```
Selecting the zone of interest:
```python
fff = pp.select_zone(fff, left=125, right=None)
```
Removing the Cosmic Rays:
```python
fff = pp.remove_CRs(fff)
```
Removing the baseline:
(don't do this on the cloud!)
```python
baseline = cc.baseline_als(fff.spectra, p=0.001)
```
Clean the spectra with the PCA:
```python
fff = pp.pca_clean(fff, n_components=21)
```

In [37]:
fff.map_params["NbSteps"]

{'MapAreaType': 'Alternating2',
 'InitialCoordinates': array([ 1000., 13985.,  -100.], dtype=float32),
 'StepSizes': array([ 33.8, -33.8,   1. ], dtype=float32),
 'NbSteps': array([209, 326,   1], dtype=uint32),
 'LineFocusSize': 0}

In [39]:
fff.spectra.shape, fff.n_y

((57684, 1015), 276)

In [34]:
fff.spectra = fff.spectra.reshape(fff.n_y, fff.n_x, -1)[50:,:,:].reshape(-1, fff.npoints)
fff.n_y -= 50

In [36]:
sns.set_style("dark")
show1 = vis.ShowSelected(fff)
show1.fig.set_size_inches(8, 6)

ValueError: cannot reshape array of size 58549260 into shape (326,209,1015)

In [10]:
fff.show_grid(alpha=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fff = pp.order(fff)


In [12]:
# =============================================================================
#                                     PCA...
# =============================================================================
fff = pp.pca_clean(fff, n_components=21)

In [13]:
fff.spectra /= np.sum(fff.spectra, axis=-1, keepdims=True)
mix, components, reconstucted_spectra = cc.deconvolute_nmf(fff, 2, max_iter=133)

# norma = np.linalg.norm(components, axis=-1, keepdims=True)
norma = np.sum(components, axis=-1)[:, None]
components /= norma
mix *= norma.T

/home/dejan/anaconda3/envs/Raman/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 133 reached. Increase it to improve convergence.
  warnings.warn(


In [14]:
_n_components = len(components)

spectra_reconstructed = np.dot(mix, components)
_mix_reshaped = mix.reshape(fff.n_y, fff.n_x, _n_components)

col_norm = colors.Normalize(vmin=0, vmax=_n_components)
color_set = ScalarMappable(norm=col_norm, cmap="brg")

# infer the number of subplots and their disposition from n_components
fi, _ax = plt.subplots(int(np.floor(np.sqrt(_n_components))),
                       int(np.ceil(_n_components /
                                   np.floor(np.sqrt(_n_components))
                                   )))
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]
for _i in range(_n_components):
    _ax[_i].plot(fff.x_values, components[_i].T, color=color_set.to_rgba(_i))
    _ax[_i].set_title(f'Component {_i}')
    _ax[_i].set_yticks([])
try:
    fi.text(0.5, 0.04,
            f"{params['XlistDataType']} recordings"
            f"in {params['XlistDataUnits']} units",
            ha='center')
except:
    pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# %%
# =============================================================================
#                       Plotting the main plot...(heatmap)
# =============================================================================
_start_pos = 0
_n_fig_rows = int(np.floor(np.sqrt(_n_components)))
_n_fig_cols = int(np.ceil(_n_components / np.floor(np.sqrt(_n_components))))

fig = plt.figure()
gs = gridspec.GridSpec(_n_fig_rows+1, _n_fig_cols, left=0.05, right=0.98, wspace=0.05)
_ax=[]
for i in range(_n_fig_rows):
    for j in range(_n_fig_cols):
        _ax.append(fig.add_subplot(gs[i, j]))
if _n_components <= 1:
    _ax = [_ax]
ax_spectra = fig.add_subplot(gs[_n_fig_rows,:])
spectra_flag = False
def onclick(event):
    '''Double-clicking on a pixel will show the (cleaned) spectrum
    corresponding to that pixel, as well as its deconvolution on the components
    and again the reconstruction for visual comparison'''
    global ax_spectra, spectra_flag, sp_l, rs_l, c_l
    if event.inaxes:
        x_pos = round(event.xdata)
        y_pos = round(event.ydata)
        broj = round(y_pos*fff.n_x + x_pos)
        spec_num = np.round(y_pos*fff.n_x - _start_pos + x_pos)
        if spectra_flag:
            if event.button!=1:
                sp_l.set_ydata(fff.spectra[spec_num])
                sp_l.set_label(f'(cleaned) spectrum n°{broj}')
                rs_l.set_ydata(spectra_reconstructed[broj])
                for k in range(_n_components):
                    c_l[k].set_ydata(components[k]*mix[broj][k])
                    c_l[k].set_label(f'C {k} = {100*mix[broj][k]/(np.sum(mix[broj])):.1f}%)')
 
                ax_spectra.relim()
                ax_spectra.legend()

        elif event.button!=1:
            spectra_flag = True
            sp_l, = ax_spectra.plot(fff.x_values, fff.spectra[spec_num], 'o', alpha=0.3,
                       label=f'(cleaned) spectrum n°{broj}')
            rs_l, = ax_spectra.plot(fff.x_values, spectra_reconstructed[broj], '--k',
                    label='reconstructed spectrum')
            c_l = []
            for k in range(_n_components):
                c_l.append(*ax_spectra.plot(fff.x_values, components[k]*mix[broj][k],
                        color=color_set.to_rgba(k),
                        label=f'C {k} = {100*mix[broj][k]/(np.sum(mix[broj])):.1f}%)'))


            ax_spectra.set_title(f'deconvolution of the spectrum from: '
                         f'line {y_pos} & column {x_pos}')
            ax_spectra.legend()
            fig.show()
    else:
        pass

for _i in range(_n_components):
    sns.heatmap(_mix_reshaped[:, :, _i], ax=_ax[_i], cmap="jet", annot=False)
#    _ax[_i].set_aspect(_s_y/_s_x)
    _ax[_i].set_title(f'Component {_i}', color=color_set.to_rgba(_i),
                      fontweight='extra bold')
fig.suptitle('Heatmaps showing the abundance of individual components'
             ' throughout the scanned area.');
fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9

In [68]:
ms = pd.read_csv(os.path.join(folder, "Microsonde", "CSSC_1200_20.txt"), delimiter="\t", encoding="iso-8859-1")

In [57]:
ms

,position (µm),Na2O (wt%),CaO (wt%),Al2O3 (wt%),SiO2 (wt%),Total,Na2O /100 (wt%),CaO /100 (wt%),Al2O3 /100 (wt%),SiO2 /100 (wt%),Total /100,Na2O /100 (mol%),CaO /100 (mol%),Al2O3 /100 (mol%),SiO2 /100 (mol%),Total /100.1
0,2320,18.76,11.07,5.40,64.74,99.97,18.77,11.07,5.40,64.76,100.00,18.56,12.11,3.25,66.08,100.0
1,2270,18.73,11.32,5.30,64.41,99.76,18.78,11.35,5.31,64.56,100.00,18.56,12.40,3.19,65.85,100.0
2,2220,18.76,11.34,5.36,64.19,99.65,18.83,11.38,5.38,64.42,100.00,18.62,12.44,3.23,65.71,100.0
3,2170,19.01,11.05,5.21,64.70,99.97,19.02,11.05,5.21,64.72,100.00,18.80,12.08,3.13,65.99,100.0
4,2120,18.89,11.08,5.29,64.15,99.42,19.00,11.14,5.32,64.52,99.99,18.79,12.18,3.20,65.83,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-1480,14.47,16.26,5.21,64.17,100.11,14.45,16.24,5.20,64.10,100.00,14.21,17.65,3.11,65.02,100.0
77,-1530,14.42,16.19,5.14,63.68,99.42,14.50,16.28,5.17,64.05,100.01,14.26,17.69,3.09,64.96,100.0
78,-1580,14.41,16.11,5.26,64.42,100.21,14.38,16.08,5.25,64.29,99.99,14.15,17.48,3.14,65.24,100.0
79,-1630,14.34,16.06,5.31,63.60,99.31,14.44,16.17,5.35,64.04,100.00,14.21,17.59,3.20,65.01,100.0
